In [1]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import EngFormatter

home = os.getcwd()
os.chdir(r"C:\Users\Tim\PycharmProjects\parrot")
import parrot
os.chdir(home)

In [2]:
os.chdir("2024-06-26/")

In [3]:
def read_boxcar(file_name, 
                boxcar_path = "boxcars/0/sample",
                position_path = "demods/0/sample.auxin0"):
    with h5py.File(file_name, 'r') as f:
        trace_ids = list(f.keys())
        # Read-in first trace
        trace_id = trace_ids[0]
        # Read out device name, e.g. dev2179
        dev = list(f[f"{trace_id}"].keys())
        # Take first element from list (there should only be one device)
        dev = dev[0]
        # Connect to path
        path = f"{trace_id}/{dev}/"
        # Read-in timebase
        timebase = f[path + boxcar_path].attrs["timebase"]
        # Read-in time array
        time = f[path + boxcar_path + "/timestamp"][:]
        # Read-in position and THz signal
        position = f[path + position_path + "/value"][:]
        signal = f[path + boxcar_path + "/value"][:]
        # Keep only values, which are not NaN
        time = time[~np.isnan(signal)]
        position = position[~np.isnan(signal)]
        signal = signal[~np.isnan(signal)]
        
        time = time[~np.isnan(position)]
        signal = signal[~np.isnan(position)]
        position = position[~np.isnan(position)]
        # Concert timesamples to time in [s]
        time = time * timebase
        # Start measurement at 0 s
        time -= np.min(time)
        data = {"time" : time,
                "position" : position,
                "signal" : signal}
    return data

In [4]:
data = read_boxcar('100kHz-0,5GaP_newmount_THz_00000.h5')
data

{'time': array([0.00000000e+00, 8.00088892e-05, 1.60017778e-04, ...,
        4.09806329e+01, 4.09807129e+01, 4.09807929e+01]),
 'position': array([-0.15930176, -0.15941894, -0.15958659, ..., -0.07233243,
        -0.07259592, -0.07226061]),
 'signal': array([0.16449105, 0.16454124, 0.16488772, ..., 0.15802646, 0.15744706,
        0.15810785])}

In [5]:
peakpeak_position = np.max(data["position"]) - np.min(data["position"])
peakpeak_position # [V]

0.2303977683186531

If 1 V == 20 mm position change of the stage, resulting in a light path change of 40 mm. 40 mm corresponds to (40 mm) / c $\approx$ 2*66 ps. That means, the scaling factor for the data is:

2*6.6e-11 s / 1 V

In [42]:
my_scale = 2 * 6.6e-11 / 1
my_scale

1.32e-10

In [43]:
EngFormatter("s")(peakpeak_position * my_scale)

'30.4125 ps'

In [34]:
fig, ax = plt.subplots()
ax.plot(data["time"], data["position"], color="tab:blue", alpha=0.8)
ax2 = ax.twinx()
ax2.plot(data["time"], data["signal"], color="tab:orange", alpha=0.8)
ax.grid(True)
ax.xaxis.set_major_formatter(EngFormatter("s"))
ax.yaxis.set_major_formatter(EngFormatter("V"))
ax2.yaxis.set_major_formatter(EngFormatter("V"))
plt.show(block=False)

KeyError: 'time'

No sample was lost during recording, otherewise there would have been a bigger time step

In [9]:
np.all(np.isclose(np.diff(data["time"]), np.diff(data["time"])[0]))

True

In [44]:
light = read_boxcar('100kHz-0,5GaP_newmount_THz_00000.h5')
dark1 = read_boxcar('100kHz-0,5GaP_newmount_dark1_00000.h5')
dark2 = read_boxcar('100kHz-0,5GaP_newmount_dark2_00000.h5')

In [45]:
data = parrot.process.thz_and_two_darks(light, 
                                        dark1, 
                                        dark2, 
                                        scale=scale, 
                                        debug=True)

[INFO] parrot.process.cut_data: Creating matrix with (4096, 102). Starting interpolation for all traces...
[INFO] parrot.process.cut_data: Creating matrix with (4096, 124). Starting interpolation for all traces...
[INFO] parrot.process.cut_data: Creating matrix with (4096, 96). Starting interpolation for all traces...


Optimization terminated successfully.
         Current function value: 40.170602
         Iterations: 11
         Function evaluations: 22


In [46]:
data.keys()

dict_keys(['light', 'dark1', 'dark2'])

In [47]:
data = parrot.post_process_data.correct_systematic_errors(data)
#data = post_obj.cut_data(time_start=-5.2e-12)
data = parrot.post_process_data.window(data)
data = parrot.post_process_data.pad_zeros(data)

[INFO] parrot.process.post_process_data: Found segment above 0.5 (rel. amplitude), from 
794.554 GHz to 860.767 GHz = 33.1064 GHz
[INFO] parrot.process.post_process_data: Found segment above 0.5 (rel. amplitude), from 
893.873 GHz to 960.086 GHz = 33.1064 GHz
[INFO] parrot.process.post_process_data: Found segment above 0.5 (rel. amplitude), from 
993.193 GHz to 1.15872 THz = 132.426 GHz
[INFO] parrot.process.post_process_data: Found segment above 0.5 (rel. amplitude), from 
1.19183 THz to 1.75464 THz = 529.703 GHz
[INFO] parrot.process.post_process_data: Found segment above 0.5 (rel. amplitude), from 
1.78775 THz to 2.0526 THz = 231.745 GHz
[INFO] parrot.process.post_process_data: Found segment above 0.5 (rel. amplitude), from 
2.0857 THz to 2.15192 THz = 33.1064 GHz
[INFO] parrot.process.post_process_data: Found segment above 0.5 (rel. amplitude), from 
2.18502 THz to 2.25124 THz = 33.1064 GHz


In [49]:
parrot.plot.simple_multi_cycle(data)
plt.show(block=False)

In [59]:
parrot.plot.extended_multi_cycle(data)

{'light': {'scale': 6.6e-11,
  'delay_value': 1.6259765625,
  'position': array([-0.163528  , -0.16351457, -0.16349414, ..., -0.1635747 ,
         -0.16358772, -0.16359644]),
  'signal': array([ 0.00257722,  0.00180253,  0.00359   , ...,  0.00246689,
          0.00247364, -0.0001059 ]),
  'number_of_traces': 102,
  'interpolation_resolution': 2048,
  'trace_cut_index': array([   996,   1999,   2997,   3997,   4994,   5995,   6995,   7993,
           8995,   9994,  10995,  11994,  12994,  13995,  14992,  15991,
          16993,  17994,  18994,  19994,  20993,  21996,  22994,  23990,
          24993,  25992,  26991,  27991,  28990,  29993,  30991,  31993,
          32992,  33990,  34993,  35990,  36993,  37991,  38991,  39990,
          40989,  41993,  42989,  43987,  44988,  45991,  46988,  47991,
          48988,  49987,  50991,  51988,  52989,  53990,  54987,  55987,
          56988,  57991,  58989,  59991,  60985,  61987,  62984,  63983,
          64986,  65985,  66987,  67986,  6898